In [1]:
from loader_ref import ExcelLoader

In [2]:
from language_ref import Language

In [3]:
loader=ExcelLoader("/workspaces/de.uke.iam.automapping/src/automapping/VM_Fragebögen_HCHS_10000_20210708.xlsx", 'Kurzname', 'Langname')

In [4]:
sample = loader.load(Language.GERMAN)[0]

In [5]:
from translator_ref import HuggingFace

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model_translator = HuggingFace(Language.GERMAN, Language.ENGLISH)

In [7]:
sample = model_translator.translate(sample)

In [8]:
print(sample)

Sample(id='E0', content='Have you been feeding yourself meatless in the past 12 months?', language=<Language.ENGLISH: 'en'>, concepts=[])


In [9]:
from preprocessor_ref import SpacyPreprocessor

In [10]:
model_spacy=SpacyPreprocessor(["lowercase", "stopwords", "punctuation", "lemmatization"])

In [11]:
model_spacy.transform(sample)

Sample(id='E0', content='feeding meatless past 12 months', language=<Language.ENGLISH: 'en'>, concepts=[])

In [12]:
sample

Sample(id='E0', content='feeding meatless past 12 months', language=<Language.ENGLISH: 'en'>, concepts=[])

## Concepts

In [13]:
from concepts_ref import Concepts

In [14]:
import pandas as pd

In [15]:
concepts = pd.read_csv('/workspaces/de.uke.iam.automapping/src/automapping/CONCEPT.csv', on_bad_lines="skip", delimiter="\t", low_memory=False)

In [16]:
synonyms = pd.read_csv('/workspaces/de.uke.iam.automapping/src/automapping/CONCEPT_SYNONYM.csv', on_bad_lines="skip", delimiter="\t", low_memory=False)

In [17]:
vocabulary = pd.read_csv('/workspaces/de.uke.iam.automapping/src/automapping/VOCABULARY.csv',on_bad_lines='skip', delimiter='\t')

In [18]:
concepts = Concepts.concatenate_concept_with_their_synonyms(concepts, synonyms, vocabulary, 'SNOMED')


/workspaces/de.uke.iam.automapping/src/automapping_ref/concepts_ref.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concepts["concept_name"] = (


In [19]:
from mapper_ref import TfIdf

In [20]:
model_tfidf = TfIdf(concepts)


In [23]:
sample = model_tfidf(sample)

In [ ]:
data_list = [{
    "id": sample.id,
    "content": sample.content,
    "language": sample.language.value,  # Assuming language is Enum
    "concepts": [concept.__dict__ for concept in sample.concepts]  # Converting each concept into dictionary
} for sample in self._detections]
# Create DataFrame from the list
df_with_results = pd.DataFrame(data_list)